# 3 Clean and Noisy Color Bases

## 3.1 Imports & Constants

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from sklearn import feature_extraction

from library import generator

%config InlineBackend.figure_format='retina'

In [2]:
DUCK = 'images/duck.jpg'

## 3.2 Utilities

In [3]:
def read_color_image(address):
    image = Image.open(address)
    pixels = np.array(image)
    pixels = pixels.astype(np.float64)
    return pixels

def to_color_patches(pixels, patch_size):
    slices = []
    channels = pixels.shape[-1]
    for channel in range(channels):
        current = pixels[:,:,channel]
        shaped_patches = feature_extraction.image.extract_patches_2d(current, patch_size)
        patches = np.reshape(shaped_patches, (len(shaped_patches), -1)).T
        slices.append(patches)
    return np.vstack(slices)

def patch_to_image(pixels, patch_size, channels):
    channel_patches = np.split(pixels, channels)
    for channel in range(channels):
        channel_patches[channel] = np.reshape(channel_patches[channel], (patch_size, patch_size))
    patch = np.dstack(channel_patches)
    return patch

def normalize_image(pixels):
    pixels = np.maximum(pixels, 0)
    pixels = np.minimum(pixels, 255)
    return pixels / 255

PIXEL_MAX = 255

def mean_squared_error(first, second):
    assert first.shape == second.shape
    error = np.mean((first - second) ** 2)
    return error

def psnr(first, second):
    error = mean_squared_error(first, second)
    if error == 0.0:
        return float('inf')
    return 20 * np.log10(PIXEL_MAX) - 10 * np.log10(error)

## 3.3 Figure Components

In [4]:
duck = read_color_image(DUCK)
noisy_duck = duck + np.random.normal(scale=20, size=duck.shape)

CHANNELS = 3
PATCH_SIZE = 8

clean_patches = to_color_patches(duck, (PATCH_SIZE, PATCH_SIZE))
noisy_patches = to_color_patches(noisy_duck, (PATCH_SIZE, PATCH_SIZE))

In [5]:
### CLEAN DUCK

fig = plt.figure(figsize=(64, 64))
plt.imshow(normalize_image(duck))
plt.axis('off')
fig.savefig('03-duck.pdf', bbox_inches='tight')
plt.close()

In [6]:
### NOISY DUCK

fig = plt.figure(figsize=(64, 64))
plt.imshow(normalize_image(noisy_duck))
plt.axis('off')
fig.savefig('03-noisy-duck.pdf', bbox_inches='tight')
plt.close()

In [7]:
## CLEAN DUCK BASES

ITERATIONS = 100

updates = generator.get_dictionary_learning_iterates(clean_patches)
clean_dictionary = next(itertools.islice(updates, ITERATIONS, None))
clean_dictionary = clean_dictionary.T

clean_encoding = clean_dictionary.T @ clean_patches

In [8]:
norms = np.abs(clean_encoding)
norms = np.sum(norms, axis=1)
all_indices = list(range(len(norms)))
all_indices.sort(key=lambda row: norms[row], reverse=True)

sum_signs = np.sum(clean_encoding, axis=1)
sum_signs = np.sign(sum_signs)

ROWS, COLS = 3, 4

fig, axs = plt.subplots(ROWS, COLS, figsize=(64, 48))
plt.subplots_adjust(left=None, right=None, bottom=None, top=None, wspace=0.05, hspace=0.05)
for index, ax in zip(all_indices, axs.flat):
    base = clean_dictionary[:,index] * sum_signs[index]
    base = base - base.min()
    base = base / base.max()
    base = patch_to_image(base, PATCH_SIZE, CHANNELS)
    
    ax.imshow(base)
    ax.axis('off')
fig.savefig('03-clean-bases.pdf', bbox_inches='tight')
plt.close()

In [9]:
## NOISY DUCK BASES

ITERATIONS = 100

updates = generator.get_dictionary_learning_iterates(noisy_patches)
noisy_dictionary = next(itertools.islice(updates, ITERATIONS, None))
noisy_dictionary = noisy_dictionary.T

noisy_encoding = noisy_dictionary.T @ noisy_patches

In [10]:
norms = np.abs(noisy_encoding)
norms = np.sum(norms, axis=1)
all_indices = list(range(len(norms)))
all_indices.sort(key=lambda row: norms[row], reverse=True)

sum_signs = np.sum(noisy_encoding, axis=1)
sum_signs = np.sign(sum_signs)

ROWS, COLS = 3, 4

fig, axs = plt.subplots(ROWS, COLS, figsize=(64, 48))
plt.subplots_adjust(left=None, right=None, bottom=None, top=None, wspace=0.05, hspace=0.05)
for index, ax in zip(all_indices, axs.flat):
    base = noisy_dictionary[:,index] * sum_signs[index]
    base = base - base.min()
    base = base / base.max()
    base = patch_to_image(base, PATCH_SIZE, CHANNELS)
    
    ax.imshow(base)
    ax.axis('off')
fig.savefig('03-noisy-bases.pdf', bbox_inches='tight')
plt.close()

## 3.4 Statistics

In [11]:
noise_stdev = 20
snr_ratio = duck.mean() / noise_stdev
print('SNR:', snr_ratio)

SNR: 6.561430358886719


In [18]:
TOP_BASES = 20

norms = np.abs(clean_encoding)
norms = np.sum(norms, axis=1)
clean_priorities = list(range(len(norms)))
clean_priorities.sort(key=lambda row: norms[row], reverse=True)

norms = np.abs(noisy_encoding)
norms = np.sum(norms, axis=1)
noisy_priorities = list(range(len(norms)))
noisy_priorities.sort(key=lambda row: norms[row], reverse=True)

In [23]:
base_angles = np.zeros((TOP_BASES, TOP_BASES))

for row in range(TOP_BASES):
    for col in range(TOP_BASES):
        base_angles[row][col] = np.abs(noisy_dictionary[:,noisy_priorities[row]] @ \
                                       clean_dictionary[:,clean_priorities[col]])
        
top_angles = []
for index in range(TOP_BASES):
    top_angles.append(max(base_angles[index]))

values = np.percentile(top_angles, [0, 25, 50, 75, 100])
print(values)

[0.25095974 0.97816219 0.98908451 0.99712264 0.99999959]
